In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

keyvault = ws.get_default_keyvault()
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f27fe043670>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [6]:
import openai
# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2023-03-15-preview"
openai.api_key = aoai_key

In [ ]:
"""import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version ="2023-03-15-preview"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")"""

## 1. Understand the AOAI Models' capabilities. Start with the latest model, prove your idea , then test with smaller models. 
Model size is critical for better performance.

## 2. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

** Length ** control (specify desired output length e.g.: number of words)

** Tone ** control (e.g.: polite, passionate, professional, technical, funny, casual, serious etc.)

** Style ** control (e.g.: in the style of Shakespeare, JK Rowling, Nelson Mandela etc.)

** Audience ** control (e.g.: a 5-year-old can understand etc)

** Context ** control (e.g.: news, novel, textbook, report, white paper, blog etc.)

In [4]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response['choices'][0]['message']['content']

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()


In [11]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = " Write an inspiring poem about GSK"
# Write a 2 paragraph inspiring poem about GSK
# Write a 2 paragraph inspiring poem focussing on innovation, vaccines, medicines of GSK in a funny way
# Write a 2 paragraph inspiring poem focussing on innovation, vaccines, medicines of GSK in the style of Shakespeare
# Write a 2 paragraph inspiring poem which can be understood by 5 years-old child focussing on innovation, vaccines, medicines of GSK

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)  

[SYSTEM]
You are a helpful assistant.

[USER]
 Write a 2 paragraph inspiring poem focussing on innovation, vaccines, medicines of GSK in a funny way

[ASSISTANT]
In the land of innovation, where GSK plays,
They twirl and swirl in a scientific ballet.
Vaccines and medicines, their craft and their art,
Healing the world, playing their part.

They dance with molecules, in labs they do jest,
Creating cures, always giving their best.
From a common cold to a stubborn flu,
They've got a potion, just right for you!



## 3. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [12]:
text = f"""
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n \
In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
broadly sharing benefits and the need to prioritize safety. \
Microsoft shares this vision and our values, and our partnership is instrumental to our progress.
"""

In [15]:
prompt = f"""
Summarize the text delimited by hashtags as a bullet point list of the most important points.
###{text}###
"""

In [18]:
response = openai.Completion.create(
            deployment_id="gpt-35-turbo",
            prompt=prompt,
            max_tokens=80,
            temperature=0,
            stop="Q"
        )

print(response['choices'][0]['text'])

- OpenAI and Microsoft are extending their partnership.
- Microsoft has invested multi-billion dollars in OpenAI.
- OpenAI is a capped-profit company and is governed by the OpenAI non-profit.
- OpenAI's mission is to ensure advanced AI benefits all of humanity.
- OpenAI's partnership with Microsoft is instrumental to their progress.




In [37]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

- OpenAI and Microsoft are extending their partnership with a multi-year, multi-billion dollar investment from Microsoft.
- This follows Microsoft's previous investments in OpenAI in 2019 and 2021.
- The investment will support OpenAI's independent research and development of safe, useful, and powerful AI.
- OpenAI remains a capped-profit company governed by the OpenAI non-profit, ensuring the benefits of AI are shared broadly and safety is prioritized.
- Microsoft shares OpenAI's vision and values, making the partnership key to their progress.


## 4. Articulate the desired output format through examples

In [12]:
prompt=f"""Extract the entities mentioned in the text below. 
First extract all company names, then extract all years, 
then extract specific topics which fit the content and finally extract general overarching themes\n\n 
Desired format: 
Company names: <comma_separated_list_of_company_names> 
Years: 
Specific topics:
General themes: 
### Text:
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n\n 
###
"""


In [13]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

Company names: OpenAI, Microsoft 
Years: 2019, 2021
Specific topics: Partnership extension, Multi-billion dollar investment, Independent research, AI development
General themes: Business partnerships, Investments, Artificial Intelligence, Research and Development


## 5.Start with zero-shot, then few-shot (example)

In [16]:
prompt_zero=f"""Extract most important keywords from the corresponding texts below.\n\n 

###Text: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords:"""

In [17]:
prompt_few=f"""Extract most important keywords from the corresponding texts below.\n\n 
### Text 1: 
Stripe provides APIs that web developers can use to integrate 
payment processing into their websites and mobile applications. \n
Keywords 1: Stripe, payment processing, APIs, web developers, websites 
### 

###Text 2: 
OpenAI has trained cutting-edge language models that are very good at understanding 
and generating text. Our API provides access to these models and can be used to solve virtually 
any task that involves processing language. \n
Keywords 2: OpenAI, language models, text processing, API.
### 

###Text 3: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords 3:"""

In [18]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt_zero

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

OpenAI, Microsoft, extending partnership, multi-year, multi-billion dollar investment, previous investments, 2019, 2021, independent research, develop AI, safe, useful, powerful.


In [19]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt_few

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful.


## 6.Instead of just saying what not to do, say what to do instead

In [20]:
base_system_message= f"""You are an agent trying to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. 
Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n"""
system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I can’t log in to my account."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

I'm sorry to hear that you're having trouble logging in. This could be due to a variety of reasons like incorrect username or password, an issue with your browser, or a problem with your account. 

Firstly, try resetting your password. You can do this by clicking on the 'Forgot Password' link on the login page. If that doesn't work, it might be a problem with your browser. Try clearing your browser's cache and cookies or try logging in from a different browser or device.

If you're still having issues, there could be a problem with your account. You can find more detailed steps on how to troubleshoot this in our help article at www.samplewebsite.com/help/faq. 

Please do not share your username, password or any other personal information here for your own security.


## 7. Divide complex tasks into sub-tasks

In [2]:
text = f"""
GSK is a fully focused biopharma company. We prioritise innovation in vaccines and specialty medicines,
maximising the increasing opportunities to prevent and treat disease. At the heart of this is our R&D focus on 
the science of the immune system, human genetics, genomics and advanced technologies, and our world-leading capabilities 
in vaccines and medicines development. We focus on four therapeutic areas: infectious diseases, HIV, respiratory/immunology, and oncology,
"""
# example 1
prompt = f"""
Perform the actions below by separating your answers with line breaks. 
1 - Summarize the following text below with 1 sentence.
2 - Translate the summary into Turkish.
3 - List each company name in the Turkish summary.
4 - Output a json object that contains the following:
keys: turkish_summary, turkish_company_names.

###
Text:
{text} 
###
"""

In [7]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

1 - GSK, aşılar ve özel ilaçlar üzerine yeniliklere öncelik veren, hastalıkları önleme ve tedavi etme fırsatlarını maksimize eden, bağışıklık sistemi, insan genetiği, genomikler ve ileri teknolojiler üzerine odaklanan, aşı ve ilaç geliştirme konusunda dünya lideri kapasitelere sahip bir biyofarma şirketidir.

2 - GSK, aşı ve özel ilaçlarda yeniliklere öncelik veren, hastalıkları önlemek ve tedavi etmek için artan fırsatları maksimize eden, bağışıklık sistemi, insan genetiği, genomlar ve ileri teknolojiler üzerinde araştırma ve geliştirme faaliyetlerine odaklanan, aşı ve ilaç geliştirme konusunda dünya lideri yeteneklere sahip bir biyofarmasötik şirketidir.

3 - GSK

4 - 
{
"turkish_summary": "GSK, aşı ve özel ilaçlarda yeniliklere öncelik veren, hastalıkları önlemek ve tedavi etmek için artan fırsatları maksimize eden, bağışıklık sistemi, insan genetiği, genomlar ve ileri teknolojiler üzerinde araştırma ve geliştirme faaliyetlerine odaklanan, aşı ve ilaç geliştirme konusunda dünya lide

## 8. Chain of Thought

The language model is prompted to generate a few intermediate reasoning steps to arrive at the final answer. 

Uses "greedy decoding" which means selecting the most likely token (word or character) at each step of the sequence generation process. At each time step, the model predicts the next token based on the previously generated tokens, and the token with the highest predicted probability is chosen as the output for that step. This process is repeated until the desired sequence length is reached or until a special end-of-sequence token is generated.

**Temp=0** is used because it uses greedy decoding. It first creates the greedy coding and then the answer.

In [34]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: The answer (arabic numerals) is
"""

In [35]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message =PROMPT_ZERO_SHOT

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=messages,
        temperature=0,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0)

response['choices'][0]['message']['content']


'11 tennis balls. \n\nExplanation: \n- Roger starts with 5 tennis balls. \n- He buys 2 cans of tennis balls, which gives him 2 x 3 = 6 more tennis balls. \n- So he now has a total of 5 + 6 = 11 tennis balls. \n- He gives 4 of them away, so he is left with 11 - 4 = 7 tennis balls.'

In [25]:
PROMPT_ZERO_SHOT_CoT = """Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Let’s think step by step.
"""

In [36]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message =PROMPT_ZERO_SHOT_CoT

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=messages,
        temperature=0, #***
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0)

response['choices'][0]['message']['content']

'1. Roger starts with 5 tennis balls.\n2. He buys 2 cans of tennis balls, which means he gets 2 x 3 = 6 more tennis balls.\n3. Now he has a total of 5 + 6 = 11 tennis balls.\n4. He gives 4 of them to his friend, which means he now has 11 - 4 = 7 tennis balls.\n\nSo, Roger has 7 tennis balls now.'

In [27]:
PROMPT_FEW_SHOT_CoT = """
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A: Elif had £10 at the beginning. When she consumed £2, 10-2=8 , £8 remains.
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A:
"""

In [37]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message =PROMPT_FEW_SHOT_CoT

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=messages,
        temperature=0,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0)

response['choices'][0]['message']['content']

'Roger had 5 tennis balls initially. He bought 2 cans of tennis balls, so he has 2 x 3 = 6 tennis balls. After giving 4 to his friend, he has 5 + 6 - 4 = 7 tennis balls left.'

** Auto-COT ** uses zero-shot-cot results just like few-shot learning for reasoning. Instead of using few-shot-cot, auto-cot can be useful and easy because you don't need to create manual examples (labels/reasonings)

In [57]:
Prompt_Auto_cot= f"""
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Let’s think step by step.Roger starts with 5 tennis balls. He buys 2 cans of tennis balls, which means he gets 2 x 3 = 6 more tennis balls. Now he has a total of 5 + 6 = 11 tennis balls.He gives 4 of them to his friend, which means he now has 11 - 4 = 7 tennis balls.So, Roger has 7 tennis balls now.
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A:
"""

In [58]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message =Prompt_Auto_cot

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=messages,
        temperature=0,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0)

response['choices'][0]['message']['content']

'Elif had £10 and consumed £2, so she now has £8 left.'

## 9. Self Consistency

Self-consistency aims "to replace the naive greedy decoding used in chain-of-thought prompting". The idea is to sample multiple, diverse reasoning paths through few-shot CoT, and use the generations to ** select the most consistent answer.**

In the chat scenarios, **Asking the model to self-verify** its own responses. Like a student double-checking their answers, the AI model cross-references its responses to maintain consistency. 

In [8]:
prompt= f"""When I was 6, my sister was half my age. Now
I’m 70 how old is my sister?"""

In [9]:
response = openai.Completion.create(
            deployment_id="textdavinci003yk",# gpt-35-turbo answers correctly
            prompt=prompt,
            max_tokens=80,
            temperature=1,
            stop="Q"
        )

print(response['choices'][0]['text'])



My sister is now 35 years old.


In [10]:
prompt2=f"""
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5
Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:"""

In [54]:
"""response = openai.Completion.create(
            deployment_id="textdavinci003yk",
            prompt=prompt2,
            max_tokens=100,
            temperature=1,
            n=4
        )

print(response['choices'][0]['text'])
print(response['choices'][1]['text'])
print(response['choices'][2]['text'])
print(response['choices'][3]['text'])"""

 When I was 6, my sister was half my age. That means she was 6 ÷ 2 = 3 years old at the time.
Now I'm 70, so she must now be 70 ÷ 2 = 35 years old. The answer is 35.
 When you were 6, your sister was half your age, so your sister was 3 years old. Now you are 70, and 70 - 6 =
64 years have passed. So your sister is now 3 + 64 = 67 years old. The answer is 67.
 6 years ago you were 6 and your sister was 6/2 = 3. That means your sister is now 70 - 6 + 3 = 67. The answer is 67.
 When you were 6, your sister was half your age. That means your sister was 6/2 = 3 years old.
Now you are 70. In the same amount of time, your sister has aged from 3 to 70 as well. So your
sister is 70 years old. The answer is 70.


In [11]:
#Calling independently to remove any bias
def call_openai(num_times, prompt, temperature):
    for i in range(num_times):
        
        response = openai.Completion.create(
            engine="textdavinci003yk",
            prompt=prompt,
            max_tokens=60,
            temperature = temperature
        )
        print(response['choices'][0]['text'])
        print("*****************************")

In [12]:
call_openai(10, prompt2, temperature = 1)

 When you were 6, your sister was 6 / 2 = 3 years old.
Now that you are 70, your sister must be 70 / 2 = 35 years old. The answer is 35.
*****************************
 When you were 6, your sister was half your age. That means she was 6 / 2 = 3 years old. Now you are 70
years old, which means your sister is 70 - 3 = 67 years old. The answer is 67.
*****************************
 When you were 6, your sister was 3 (half your age). 70 years later, your sister is 70 - 6 + 3 = 67 years old. The answer is 67.
*****************************
 She was half your age when you were 6, so she was 6 / 2 = 3 at that time. That means she is 70 - 3 = 67 now. The answer is 67.
*****************************
 We know that when you were 6, she was half your age. That means she was 3 at that time. Now you are
70 years old, so she has aged 70 - 3 = 67 years. She must be 67 years old. The answer is 67.
*****************************
 When the speaker was 6, his sister was 6/2 = 3 years old. Now the speaker is 7

## 10. Iterative approach

Prompt engineering is an iterative process. If you're unsatisfied with the AI's response, refine your prompt and try again. Analyze the results you receive and consider adjusting your prompt's context, clarity, or structure. This process of trial and error will help you better understand how the AI model interprets your prompts and allow you to fine-tune your approach.

·        Try different prompts to find what works best

·        When attempting few-shot learning, try also to include direct instructions

·        Rephrase a direct instruction set to be more or less concise, e.g.: taking a previous example and giving the next instruction without having to repeat the input

·        Try different personas keywords to see how it affects the response style

·        Use fewer or more examples in the few-shot learning

·        Co-create with AI: An example of a very useful prompt to get a good output from the LLM :

In [13]:
text=f"""
Global health and health security
We use our expertise to address the biggest health challenges for underserved 
people around the world

£1bn
investment in R&D to help 
us get ahead of infectious 
diseases in lower income 
countries

12
Global Health pipeline assets 
to address priority WHO 
diseases progressed in 2022

>30
R&D projects relevant to 
antimicrobial resistance in 
our pipeline

Our commitment
To develop novel products and technologies to treat and 
prevent priority diseases, including pandemic threat
"""

In [14]:
prompt_iterative= f""" Your task is to explain given information in a very simple way.
### Context:
{text}
"""

In [15]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt_iterative

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

We are experts in dealing with major health issues that affect people who don't have enough resources all around the world. We've put £1 billion into research and development to help us tackle infectious diseases in poorer countries. In 2022, we made progress on 12 key health projects that focus on diseases the World Health Organization has identified as priorities. We have more than 30 projects underway that are related to fighting antimicrobial resistance. Our promise is to create new products and technologies to treat and prevent important diseases, including those that could cause a pandemic.


**Issue 1:** It is long explaination.

In [17]:
prompt_iterative1= f""" Your task is to explain given information in a very simple way.
Use at most 20 words.

### Context:
{text}
"""

In [18]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt_iterative1

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

We use our skills to tackle health issues for needy people globally, investing £1bn in research, with over 30 projects ongoing.


**Issue 2:** Change the output format

In [22]:
prompt_iterative2= f""" Your task is to explain given information in a very simple way.
Use at most 20 words.

If there is a numerical value, write those in a table format and named it as "Global health and health security".
### Context:
{text}
"""

In [23]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt_iterative2

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

We're using our skills to tackle big health problems for needy people globally.

**Global health and health security**

|   |   |
|---|---|
| £1bn | Invested in R&D for infectious diseases in poorer countries |
| 12 | Global Health assets for priority diseases advanced in 2022 |
| >30 | R&D projects for antimicrobial resistance |

We're committed to creating new treatments for important diseases, including pandemics.


In [30]:
prompt_iterative3= f""" Your task is to explain given information in a very simple way.
Use at most 20 words.

If there is a numerical value, write those in a table format and named it as "Global health and health security".

Format everything as HTML that can be used in a website. 

### Context:
{text}
"""

In [32]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt_iterative3

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

```html
<h2>Global health and health security</h2>
<p>We use our expertise to solve major health issues for the underprivileged worldwide.</p>

<h3>Our commitment</h3>
<p>We aim to create new products and technologies to treat and prevent serious diseases, including pandemic threats.</p>

<h3>Global health and health security</h3>
<table>
  <tr>
    <th>Investment in R&D</th>
    <th>Global Health pipeline assets</th>
    <th>R&D projects relevant to antimicrobial resistance</th>
  </tr>
  <tr>
    <td>£1bn</td>
    <td>12</td>
    <td>>30</td>
  </tr>
</table>
```


In [33]:
from IPython.display import display, HTML
display(HTML(response))

Investment in R&D,Global Health pipeline assets,R&D projects relevant to antimicrobial resistance
£1bn,12,>30


In [35]:
prompt_iterative4= f""" Your task is to explain given information in a very simple way.
Use at most 20 words.

If there is a numerical value, write those in a table format and named it as "Global health and health security".

Format everything as HTML that can be used in a website without using html keyword. 

### Context:
{text}
"""

In [36]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt_iterative4

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

<h3>Global health and health security</h3>
<p>We use our knowledge to tackle major health issues for less privileged people globally.</p>

<table>
  <tr>
    <th>Investment in R&D</th>
    <th>Global Health Pipeline Assets</th> 
    <th>R&D Projects</th>
  </tr>
  <tr>
    <td>£1bn</td>
    <td>12</td> 
    <td>>30</td>
  </tr>
</table>

<h3>Our commitment</h3>
<p>We aim to create new products and technologies to address and prevent priority diseases, including pandemic threats.</p>


In [37]:
from IPython.display import display, HTML
display(HTML(response))

Investment in R&D,Global Health Pipeline Assets,R&D Projects
£1bn,12,>30
